# 8. Make Predictions

In [1]:
import pandas as pd
import pickle
import json
import os
from collections import defaultdict as dd

In [2]:
with open(f'../models/final_models/fixed_tfagnn.pickle', 'rb') as f:
    model = pickle.load(f)

with open('../models/feature_importance_ordering.pickle', 'rb') as f:
    feature_importance_ordering = pickle.load(f)

In [3]:
manip_type = 'NormalisedData'

csv_list = os.listdir(f'../future data/curated/{manip_type}')
csv_list.sort()

In [4]:
def predict_brownlow(csv_list, year, graph=False):
    json_dict = dict()

    tally = dd(int)

    data = pd.DataFrame()
    for file in csv_list:
        if file[-4:] != '.csv':
            continue

        game_dict = dict()

        if str(year) in file:

            round = file.split()[2]
            team1 = file.split()[3]
            team2 = file.split()[5]
            game = team1 + ' v ' + team2

            data = pd.read_csv(f'../future data/curated/{manip_type}/{file}')

            if graph:
                data['idx'] = 0

            player = data['Player']
            player = data['Player']
            pred = model.predict(
                data[list(list(feature_importance_ordering.keys())[36])+['idx']] if graph else
                data[list(feature_importance_ordering.keys())[36]])
            pred = pd.DataFrame({'player': player, 'predicted_score': pred})

            three_votes = list(pred.sort_values(
                'predicted_score', ascending=False)['player'])[0]

            two_votes = list(pred.sort_values(
                'predicted_score', ascending=False)['player'])[1]

            one_vote = list(pred.sort_values(
                'predicted_score', ascending=False)['player'])[2]

            pred = pred.sort_values('predicted_score', ascending=False)
            pred.to_csv(
                f'../presentables/raw_predicted_scores/{year}_round{round}_{game}.csv', index=False)

            game_dict[3] = three_votes
            game_dict[2] = two_votes
            game_dict[1] = one_vote

            if f'Round {round}' in json_dict:
                json_dict[f'Round {round}'][game] = game_dict
            else:
                json_dict[f'Round {round}'] = dict()
                json_dict[f'Round {round}'][game] = game_dict

            tally[three_votes] += 3
            tally[two_votes] += 2
            tally[one_vote] += 1

    return json_dict, tally

In [5]:
json_dict, tally = predict_brownlow(csv_list, 2024, graph=True)

/Applications/anaconda3/lib/python3.9/site-packages/PyTorch2Sklearn/__template__.py:488: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  torch.FloatTensor(x_list[mini_batch_number])


In [6]:
# with open('../presentables/game_by_game_prediction.json', 'w') as f:
#     json.dump(json_dict, f, indent=2)

In [7]:
tally_list = list(tally.items())
tally_list.sort(key=lambda x: x[1], reverse=True)

In [8]:
tally_df = pd.DataFrame(tally_list, columns=['Player', 'Votes'], index=[
                        i+1 for i in range(len(tally_list))])
tally_df['Ranking'] = tally_df.index
tally_df = tally_df[['Ranking', 'Player', 'Votes']]
tally_df.head(60)

,Ranking,Player,Votes
1,1,Nick Daicos,33
2,2,Marcus Bontempelli,31
3,3,Patrick Cripps,27
4,4,Lachie Neale,25
5,5,Noah Anderson,23
6,6,Zak Butters,23
7,7,Matt Rowell,21
8,8,Zachary Merrett,21
9,9,Andrew Brayshaw,20
10,10,Adam Treloar,20


In [9]:
# tally_df.to_csv('../presentables/leaderboard.csv', index=False)